In [6]:
import numpy as np
import sys, platform
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

print("Python full:", sys.version)          # e.g. '3.12.5 (main, ...)'
print("Python tuple:", sys.version_info)     # e.g. sys.version_info(major=3, minor=12, micro=5, ...)
print("Python short:", platform.python_version())  # e.g. '3.12.5'
print("Executable:", sys.executable)         # path to the kernel's python


Python full: 3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 08:03:38) [Clang 14.0.6 ]
Python tuple: sys.version_info(major=3, minor=12, micro=11, releaselevel='final', serial=0)
Python short: 3.12.11
Executable: /opt/homebrew/anaconda3/envs/qiskit2x/bin/python


In [7]:
QiskitRuntimeService.save_account(
    channel='ibm_quantum_platform',
    token='RH-RO4uf1A9A4A7LkJsTBw27L2RH4BqAIodMBiZS0QyI',
    set_as_default=True,
    overwrite=True,
)
print("Saved IBM Quantum Platform account.")


Saved IBM Quantum Platform account.


In [10]:
n_qubits = 50
 
service = QiskitRuntimeService()
print('the backends available are: ', service.backends())
# backend = service.least_busy(
    # operational=True, simulator=False, min_num_qubits=n_qubits
# )
backend = service.backends()[0]

 
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)
 
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
 
estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()
 
print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")


qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-09-20 22:03:12,256: Default instance not set. Searching all available instances.


the backends available are:  [<IBMBackend('ibm_torino')>, <IBMBackend('ibm_brisbane')>]
 > Expectation value: -0.7012987012987013
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
